### Testing our models

This notebook will load saved model and predict the image labels

In [16]:
import torch
from torchvision.transforms import transforms
from PIL import Image
from pathlib import Path
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader

In [21]:
import os

In [4]:
model = torch.load(Path('/home/ubuntu/codebase/my_github/kaggle-competitions/bengali-ai/approach_2/grapheme-root-base.pth'))

In [5]:
trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.0692, 0.0692, 0.0692], [0.2051, 0.2051, 0.2051])
    ])

In [7]:
image = Image.open(Path('/home/ubuntu/datasets/bengali-ai/testing_images/testing_224by224/Test_0.png'))

In [9]:
input = trans(image)

In [ ]:
output = model(input)

prediction = int(torch.max(output.data, 1)[1].numpy())
print(prediction)

In [25]:
class BengaliCharacterDataset(Dataset):
    """Bengali language handwritten character dataset."""
    
    def __init__(self, root_dir, transform=None):
        """
        Args:
            dataframe (string): Dataframe of parquet dataset with image data.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):        
        return len(os.listdir(self.root_dir))

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image_id = 'Train_'+str(idx)
        img_name = image_id+'.png'
        
        # columns from class_map: image_id, grapheme_root, vowel_diacritic, consonant_diacritic, grapheme
        # img_label = train_labels.loc[train_labels['image_id'] == image_id, 'grapheme_root'].to_numpy()[0] 
        # added to.numpy()[0] to remove index number
                
        img_path = os.path.join(self.root_dir,img_name)
        image = io.imread(img_path)

        # sample = {'image': img_data, 'img_label': img_label}
        
        if self.transform:
            image = self.transform(im.fromarray(image).convert('RGB'))
            # sample = {'img_label': img_label, 'image': img_data}

        return image

In [28]:
# init the test loader
test_loader = torch.utils.data.DataLoader(
    BengaliCharacterDataset('/home/ubuntu/datasets/bengali-ai/testing_images/testing_224by224/',
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.0692,0.0692,0.0692), (0.2051,0.2051,0.2051))
                   ])),
    batch_size=2, shuffle=True
)

In [ ]:
single_loaded_img = test_loader.dataset.data[0]
single_loaded_img = single_loaded_img.to(device)
single_loaded_img = Variable(single_loaded_img)
out_predict = model(single_loaded_img)

In [29]:
test_loader.dataset.get